# BGG review score prediction: deep learning approach (YH)

In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../data/clean_data.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,user,rating,comment,ID,name
0,4108284,4108284,PegasusGamesNYC,2.0,Played it once and didn't like it,9209,Ticket to Ride
1,9412682,9412682,captaincomic,4.0,Several things to dislike: Player elimination;...,24068,Shadow Hunters
2,3448962,3448962,jblomquist,4.0,Seems like an ok game but it doesn't really wo...,150376,Dead of Winter: A Crossroads Game
3,10640158,10640158,Rothro,3.0,Flux and Munchkin mixed but worse. Way way way...,234190,Unstable Unicorns
4,14938520,14938520,armourer84,3.0,The game open the player to be sieged by group...,6281,Eketorp


In [5]:
X = df['comment']
y = df['rating']

In [6]:
X

0                        Played it once and didn't like it
1        Several things to dislike: Player elimination;...
2        Seems like an ok game but it doesn't really wo...
3        Flux and Munchkin mixed but worse. Way way way...
4        The game open the player to be sieged by group...
                               ...                        
99995    as you get a little more familiar with it you ...
99996    This is a fun, fast, and clever game.  It feel...
99997                                              Amazing
99998    Nice one :) First off, i bought it to play sol...
99999    Kids rating  My daughter enjoys this one a lot...
Name: comment, Length: 100000, dtype: object

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

### Let's tokenize the vocabulary 
tk = Tokenizer()
tk.fit_on_texts(X)
vocab_size = len(tk.word_index)
print(f'There are {vocab_size} different words in your corpus')
X_token = tk.texts_to_sequences(X)

ModuleNotFoundError: No module named 'tensorflow'

In [9]:
X_token.shape

NameError: name 'X_token' is not defined